In [1]:
import pandas as pd

In [2]:
filename = 'GSE_2018.csv'

In [3]:
df = pd.read_csv(filename)

In [4]:
# clean time and convert to datetime
df['time_period'] = df['time_period'].str[1:-1]
df['datetime'] = df['service_date'] + " " + df['time_period']
df['datetime'] = pd.to_datetime(df['datetime'])

In [5]:
# get entry total for each station for each date by hour
df = df.groupby(['datetime', 'stop_id'])['gated_entries'].sum().reset_index()
df['gated_entries'] = df['gated_entries'].astype(int)
df['date'] = df['datetime'].dt.date
df['hour'] = df['datetime'].dt.hour
df = df.groupby(['date', 'hour', 'stop_id'])['gated_entries'].sum().reset_index()

In [6]:
# define am rush hour as 6-10, define pm rush hour as 3-7
df_am = df.loc[(df['hour'] >= 6) & (df['hour'] <= 10), :]
df_am = df_am.groupby(['date', 'stop_id'])['gated_entries'].sum().reset_index()
df_am['am_rush'] = df_am['gated_entries']
df_am = df_am.loc[:, ['date', 'stop_id', 'am_rush']]

df_pm = df.loc[(df['hour'] >= 15) & (df['hour'] <= 19), :]
df_pm = df_pm.groupby(['date', 'stop_id'])['gated_entries'].sum().reset_index()
df_pm['pm_rush'] = df_pm['gated_entries']
df_pm = df_pm.loc[:, ['date', 'stop_id', 'pm_rush']]

In [7]:
# get entry total for each station by date
df_day = df.groupby(['date', 'stop_id'])['gated_entries'].sum().reset_index()
df_day['full_day'] = df_day['gated_entries']
df_day = df_day.loc[:, ['date', 'stop_id', 'full_day']]

In [8]:
# combine everything into one dataframe
df_full_day = df_am.merge(df_pm, left_on=['date', 'stop_id'], right_on=['date', 'stop_id'])
df_full_day = df_full_day.merge(df_day, left_on=['date', 'stop_id'], right_on=['date', 'stop_id'])

In [9]:
df_full_day

,date,stop_id,am_rush,pm_rush,full_day
0,2015-01-01,place-alfcl,477,847,2775
1,2015-01-01,place-andrw,426,631,1765
2,2015-01-01,place-aport,725,1121,3598
3,2015-01-01,place-aqucl,164,1270,2411
4,2015-01-01,place-armnl,188,1002,2117
...,...,...,...,...,...
22311,2015-12-31,place-welln,2281,1849,5648
22312,2015-12-31,place-wimnl,577,386,1434
22313,2015-12-31,place-wlsta,1758,755,3442
22314,2015-12-31,place-wondl,2344,1377,5111
